In [ ]:
%pip install ANNarchy
%pip install gymnasium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests

url = "https://github.com/bastianloyola/NEAT-ANNarchy/raw/19ff200696d4973694ea47f55aa4404116827fa3/neatannarchy.py"
file_name = "neatannarchy.py"
response = requests.get(url)

if response.status_code == 200:
    with open(file_name, "wb") as file:
        file.write(response.content)
    print(f"Archivo descargado y guardado como '{file_name}'.")
else:
    print(f"Error al descargar el archivo: {response.status_code}")


/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Archivo descargado y guardado como 'neatannarchy.py'.


In [ ]:
from ANNarchy import *
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import scipy.sparse
import gymnasium as gym
from scipy.special import erf

ANNarchy 4.7 (4.7.3) on linux (posix).


Para utilizar el algoritmo evolutivo se deben definir los parametros de uso, como el modelo neuronal, la red, funcion objetivo y las configuraciòn de hiperparametros

# Modelos de Neurona
Mediante annarchy definir las neuronas a utilizar, se debe generar un archivo de python llamador neuronmodel.py por eso se utiliza la sentencia "%%writefile"

Para mayor guia en la definiciòn de modelos de neurona para tu red neuronal de espiga utilizar https://annarchy.readthedocs.io/API/Neuron.html

Ejemplos:

In [ ]:
%%writefile neuronmodel.py

from ANNarchy import *

LIF = Neuron(  #I = 75
    parameters = """
    tau = 50.0 : population
    I = 0.0
    tau_I = 10.0 : population
    """,
    equations = """
    tau * dv/dt = -v + g_exc - g_inh + (I-65) : init=0
    tau_I * dg_exc/dt = -g_exc
    tau_I * dg_inh/dt = -g_inh
    """,
    spike = "v >= -40.0",
    reset = "v = -65"
)

IZHIKEVICH = Neuron(  #I = 20
    parameters="""
        a = 0.02 : population
        b = 0.2 : population
        c = -65.0 : population
        d = 8.0 : population
        I = 0.0
        tau_I = 10.0 : population
    """,
    equations="""
        dv/dt = 0.04*v*v + 5*v + 140 - u + I + g_exc - g_inh : init=-65
        tau_I * dg_exc/dt = -g_exc
        tau_I * dg_inh/dt = -g_inh
        du/dt = a*(b*v - u) : init=-14.0
    """,
    spike="v >= 30.0",
    reset="v = c; u += d"
)

Overwriting neuronmodel.py


# Definición de la función objetivo

En este apartado, se debe definir la codificaciòn de la entrada para la red neuronal y además la decodificación de la salida. En esste proceso se debe construir una función objetivo que devuelva un unico valor.




Ejemplo 1:
Problema no linear XOR
Entrada de la red, corresponde a entregar un pulso a la neurona asociada a la variable con un 1, la decodificación corresponde a si la neurona de salida emite al menos un impulso

In [ ]:
%%writefile fitness.py

from ANNarchy import *
import numpy as np

def xor(pop,Monitor,input_index,output_index,inputWeights):
    Monitor.reset()
    entradas = [(0, 0), (0, 1), (1, 0), (1, 1)]
    fitness = 0
    for entrada in entradas:
        for i, val in zip(input_index, entrada):
            if val == 1:
                pop[int(i)].I = 15.1*inputWeights[i]
            else:
                pop[int(i)].I = 0
        simulate(10.0)
        spikes = Monitor.get('spike')
        #print("spikes: ",spikes)
        #Get the output
        output = 0
        for i in output_index:
            output += np.size(spikes[i])
        #print("spike output: ",output)

        decode_output = 0
        if output > 1:
            decode_output = 1

        pop.reset()
        Monitor.reset()
        #comparar las entradas y la salida esperada con el output
        if entrada[0] ^ entrada[1] == decode_output:
            fitness += 1
    return fitness

Overwriting fitness.py


# Hiperparámetros evolutivos

Se deben definir un conjunto de hiperparametros que se utilizarán en el proceso de evolución, además se deben definir ciertos parametros de configuración relacionados a la poblaciòn, función objetivo y entre otros.

Para ello primero se debe contar con una carpeta llamada config donde se tendrá un archivo con los hiperparametros a utilizar.

In [ ]:
%mkdir -p config

A continuación una definición de los elementos contenidos en el archivo de configuración

- keep: porcentaje de población que quedará luego del proceso de eliminación
- threshold: Umbral de diferencia entre un individuo y el representante de una especie para incorporarlos a la especie en cuestion
- interSpecieRate: probabilidad de generar un entrecruzamiento entre individuos de distintas especies
- noCrossoverOff: probabilidad de generar un individuo nuevo en la población, unicamente de una mutación.
- probabilityWeightMutated: Probabilidad de que una mutación sea cambiar el peso de una conexión aleatoria
- probabilityAddNodeSmall: Probabilidad de que una mutación sea añadir un nodo en una conexión aleatoria para una red pequeña
- probabilityAddLink_small: Probabilidad de que una mutación sea añadir una conexión aleatoria para una red pequeña
- probabilityAddNodeLarge: Probabilidad de que una mutación sea añadir un nodo en una conexión aleatoria para una red grande
- probabilityAddLink_Large: Probabilidad de que una mutación sea conexión aleatoria para una red grande
- c1, c2 y c3: Factores utilizados en el calculo de un valor representativo de la red, utilizado en la especiación
- largeSize: Cantidad de neuronas necesarias para considerar a una red como grande
- numberInputs: Cantidad de neuronas de entrada en cada red
- numberOutputs: Cantidad de neuronas de salida en cada red
- n_max: Cantidad de neuronas maximas que puede tener una red
- learningRate: Valor que define el maximo que se le sumará o restará al peso de una conexión
- inputWeights: Serie de valores separados por coma, que pueden ser utilizados en la función objetivo.
- weightsRange: Valores que tendrán los pesos de las redes al crearlas. Puede ser un rango n,m (donde n < m) y se tomará valores aleatorios en dicho rango. También si se desea tener pesos de un solo valor "n", pero que sean positivos y negativos de forma aleatoria se debe definir como n,n
- function: Nombre de la función objetivo definida
- neuronModel: Nombre del modelo neuronal escogido

In [ ]:
%%writefile config/config.cfg
keep=0.49
threshold=3.251
interSpeciesRate=0.0001
noCrossoverOff=0.159
probabilityWeightMutated=0.821
probabilityAddNodeSmall=0.026
probabilityAddLink_small=0.038
probabilityAddNodeLarge=0.223
probabilityAddLink_Large=0.154
c1=1.188
c2=1.09
c3=0.481
largeSize=8
numberInputs=2
numberOutputs=1
n_max=100
learningRate=10
inputWeights=110,150
weightsRange=-20,80

Overwriting config/config.cfg


Parametros de ejecución

- func: Nombre de la función objetivo a evaluar
- neuron_model: Nombre del modelo neuronal utilizado en la red
- procesos: Cantidad de procesos utilizados en la evaluación de los individuos de la población
- evolutions: Cantidad de iteraciones del ciclo evolutivo, es decir las numero de evoluciones de la población
- population: Cantidad de genomas en la red

# Ejemplo 1 XOR

In [ ]:
func = "xor"
neuron_model = "LIF"
procesos = 2
evolutions = 2
population = 4

In [ ]:
from neatannarchy import runNEAT

#runNEAT requiere como entrada un valor que identificará la carpeta donde se guardará la información de la ejecución
#esta carpeta se guardará como results/trial-X, donde X es el valor de la entrada
#Las otras entradas son el nombre de la función de fitness, el modelo de neurona, el número de procesos, el número de evoluciones y el tamaño de la población
fitness1 = runNEAT(1, func, neuron_model, procesos, evolutions, population)
print("Fitness: ", fitness1)

Fitness:  18.01
